In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
from keras.models import load_model
import numpy as np
import os

In [46]:
def capture_samples():
    # Directory to save captured photos
    save_dir = 'Train'

    name = input("Enter your name")
    
    save_dir = save_dir + "/" + name
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Load the pre-trained face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Initialize the camera
    cap = cv2.VideoCapture(0)
    
    # Counter to keep track of captured photos
    photos_count = 0
    
    while photos_count < 600:
        # Capture frame-by-frame
        ret, frame = cap.read()
    
        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the grayscale frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=8)
    
        # If a face is detected, save the photo
        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Draw rectangle around the detected face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
                # Save the detected face
                face_img = frame[y:y+h, x:x+w]
                img_name = os.path.join(save_dir, f"photo_{photos_count}.jpg")
                cv2.imwrite(img_name, face_img)
    
                print(f"Photo {photos_count + 1} captured and saved as {img_name}")
    
                # Increment photo count
                photos_count += 1
    
        # Display the resulting frame
        cv2.imshow('Capture Photos', frame)
    
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close OpenCV windows
    cap.release()
    cv2.destroyAllWindows()
    

In [64]:
# capture_samples()

Enter your name Shreshth


Photo 1 captured and saved as Train/Shreshth\photo_0.jpg
Photo 2 captured and saved as Train/Shreshth\photo_1.jpg
Photo 3 captured and saved as Train/Shreshth\photo_2.jpg
Photo 4 captured and saved as Train/Shreshth\photo_3.jpg
Photo 5 captured and saved as Train/Shreshth\photo_4.jpg
Photo 6 captured and saved as Train/Shreshth\photo_5.jpg
Photo 7 captured and saved as Train/Shreshth\photo_6.jpg
Photo 8 captured and saved as Train/Shreshth\photo_7.jpg
Photo 9 captured and saved as Train/Shreshth\photo_8.jpg
Photo 10 captured and saved as Train/Shreshth\photo_9.jpg
Photo 11 captured and saved as Train/Shreshth\photo_10.jpg
Photo 12 captured and saved as Train/Shreshth\photo_11.jpg
Photo 13 captured and saved as Train/Shreshth\photo_12.jpg
Photo 14 captured and saved as Train/Shreshth\photo_13.jpg
Photo 15 captured and saved as Train/Shreshth\photo_14.jpg
Photo 16 captured and saved as Train/Shreshth\photo_15.jpg
Photo 17 captured and saved as Train/Shreshth\photo_16.jpg
Photo 18 capture

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the main directory containing the 'train' and 'test' folders
main_directory = "."

# Specify the training dataset directory
train_directory = main_directory + "/Train"

In [5]:
image_size = (256, 256)
batch_size = 32

In [65]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)

Found 2551 files belonging to 4 classes.
Using 2041 files for training.


In [66]:
class_data = train_dataset.class_names

In [67]:
class_data

['Aman', 'Jharna', 'Lakhan', 'Shreshth']

In [57]:
# Create a simple CNN model
from keras import models, layers

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

In [58]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
# Train the model
#uncomment to train 

history = model.fit(
    train_dataset,
    epochs=3,  # Adjust the number of epochs based on your needs
)


Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.8863 - loss: 3.7752
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - accuracy: 0.9741 - loss: 1.1275
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.9834 - loss: 1.1032


In [22]:
# model.save("modelWith6.keras")

In [8]:
from tensorflow.keras.models import load_model
# model = load_model("modelWith6.keras")

In [63]:
facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
font = cv2.FONT_HERSHEY_COMPLEX

def get_className(classNo, class_data):
    return class_data[classNo]

while True:
    success, imgOrignal = cap.read()
    faces = facedetect.detectMultiScale(imgOrignal, 1.3, 5)
    for x, y, w, h in faces:
        crop_img = imgOrignal[y:y+h, x:x+h]
        img = cv2.resize(crop_img, (256, 256))
        img = img.reshape(1, 256, 256, 3)
        prediction = model.predict(img)
        classIndex = np.argmax(prediction)
        probabilityValue = np.amax(prediction)
        
        cv2.rectangle(imgOrignal, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.rectangle(imgOrignal, (x, y), (x+w, y), (0, 255, 0), 2)
        cv2.putText(imgOrignal, str(get_className(classIndex,class_data)), (x, y-10), font, 0.75, (255, 255, 255), 1, cv2.LINE_AA)
        
        cv2.putText(imgOrignal, str(round(probabilityValue*100, 2))+"%", (180, 75), font, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow("Result", imgOrignal)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━